In [1]:
from torch_geometric.datasets import QM9
# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import logging
import time
import wandb
wandb.login()
import random

import os.path as osp

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import GRU, Linear, ReLU, Sequential
import torch.optim as optim

import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv, Set2Set, MessagePassing, global_mean_pool, aggr,GCNConv
from torch_geometric.utils import remove_self_loops

wandb: Currently logged in as: irisxinran (ntuwb). Use `wandb login --relogin` to force relogin


Each model and target combination was trained using a uniform random hyper parameter search with 50 trials. 

T was costrained to be in the range 3 ≤ T ≤ 8 (in practice, any T ≥ 3 works). 

The number of set2set computations M was chosen from the range 1 ≤ M ≤ 12. 

All models were trained using SGD with the ADAM optimizer (Kingma &Ba (2014)), with batch size 20 for 3 million steps ( 540 epochs). 

The initial learning rate was chosen uniformly between 1e−5 and 5e−4. We used a linear learning rate decay that began between 10% and 90% of the way through training and the initial learning rate l decayed to a final learning ate l ∗ F, using a decay factor F in the range [.01, 1].

The QM-9 dataset has 130462 molecules in it. We randomly chose 10000 samples for validation, 10000 samples or testing, and used the rest for training. We use the validation set to do early stopping and model selection and we eport scores on the test set. 

All targets were normalized o have mean 0 and variance 1. We minimize the mean quared error between the model output and the target, although we evaluate mean absolute error.

In [2]:
model_name = "adam_mse_nnconv_poolMaxAggr_2fc_wandb_random_new"

In [3]:
import wandb
import numpy as np 
import random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define sweep config
sweep_configuration = {
'method': 'random',
'metric': {'goal': 'minimize', 'name': 'val_loss'},
'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 16,
                               'q': 8},
                'criterion': {'value': 'MSELoss()'},
                'epochs': {'values': [20,40,60]},
                'hidden_dim_1': {'value': 64},
                'hidden_dim_2': {'value': 128},
                'hidden_dim_3': {'value': 32},
                'in_channels': {'value': 11},
                'lr': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'nnconv_aggr': {'values': ['max','mean','add']},
                'optimizer': {'value': 'adam'},
                'out_channels': {'value': 1}
              }
}

# Initialize sweep by passing in config. 
# (Optional) Provide a name of the project.
sweep_id = wandb.sweep(
  sweep=sweep_configuration, 
  project=model_name
  )

# Define the dataset and do transformation
class TargetTransform:
    def __call__(self, data):
        # Specify target.
        target = 0 # the first property is the one to be predicted
        data.y = data.y[:, target]
        return data
    
def data(bs):
    path = './datasets/QM9'
    transform = T.Compose([TargetTransform(), T.Distance(norm=False)]) # add the distance into edge attributes
    dataset = QM9(path, transform=transform)
    # Split datasets.
    torch.manual_seed(12345)
    dataset = dataset.shuffle()
    test_dataset = dataset[:10000]
    val_dataset = dataset[10000:20000]
    train_dataset = dataset[20000:]
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False) #10000
    val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False) #10000
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True) #110831
    return test_loader, train_loader, val_loader

# Define training function that takes in hyperparameter 
# values from `wandb.config` and uses them to train a 
# model and return metric
def train_one_epoch(model, train_loader, epoch, lr, optimizer): 
    # Train step
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch).flatten()
        loss = nn.MSELoss()
        loss_output = loss(output, batch.y)
        loss_output.backward()
        optimizer.step()
        total_loss += loss_output * batch.num_graphs

    average_loss = total_loss  / len(train_loader.dataset)
    
    print(f'Epoch: {epoch}, Training Loss: {average_loss:.4f}')
    logging.info(f'Epoch: {epoch}, Training Loss: {average_loss:.4f}')
    
    return average_loss

def evaluate_one_epoch(model, val_loader, epoch, lr): 
    # Validation step
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for data in val_loader:
            data = data.to(device)
            output = model(data).flatten()
            val_loss = nn.MSELoss()
            val_loss_output = val_loss(output, data.y)
            total_val_loss += val_loss_output * data.num_graphs

    average_val_loss = total_val_loss / len(val_loader.dataset)
    
    print(f'Epoch: {epoch}, Validation Loss: {average_val_loss:.4f}')
    logging.info(f'Epoch: {epoch}, Validation Loss: {average_val_loss:.4f}')
    
    return average_val_loss

# Define the GNN model
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_dim_1, hidden_dim_2, hidden_dim_3, out_channels, nnconv_aggr):
        super(GNNModel, self).__init__()
        #nn – A neural network that maps edge features edge_attr of shape [-1, num_edge_features] to shape [-1, in_channels * out_channels], e.g., defined by torch.nn.Sequential.
        nn1 = nn.Sequential(nn.Linear(5, 128), ReLU(), nn.Linear(128, in_channels * hidden_dim_1)) # maps to in_channels * out_channels
        self.conv1 = NNConv(in_channels, hidden_dim_1, nn1, aggr=nnconv_aggr) #11*64
        self.global_pool = aggr.MaxAggregation()
        self.fc1 = nn.Linear(hidden_dim_1, hidden_dim_3)
        self.fc2 = nn.Linear(hidden_dim_3, out_channels)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        x = self.conv1(x, edge_index, edge_attr).relu()
        #x = global_mean_pool(x, batch)  # size = [batch_size, hidden_channels]
        x = self.global_pool(x,batch)
        x = self.fc1(x).relu()
        x = self.fc2(x)
        return x
    
def build_model(in_channels, hidden_dim_1, hidden_dim_2, hidden_dim_3, out_channels, nnconv_aggr):
    # Initialize the model, optimizer, and loss function
    model = GNNModel(in_channels, hidden_dim_1, hidden_dim_2, hidden_dim_3, out_channels, nnconv_aggr)
    print(model)
    logging.info(model)
    return model

def build_optimizer(model, lr):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    return optimizer
    
def main():
    run = wandb.init()

    # note that we define values from `wandb.config`  
    # instead of defining hard values
    lr  =  wandb.config.lr
    bs = wandb.config.batch_size
    epochs = wandb.config.epochs
    in_channels = wandb.config.in_channels
    hidden_dim_1 = wandb.config.hidden_dim_1
    hidden_dim_2 = wandb.config.hidden_dim_2
    hidden_dim_3 = wandb.config.hidden_dim_3
    out_channels = wandb.config.out_channels
    nnconv_aggr = wandb.config.nnconv_aggr
    criterion = wandb.config.criterion
    
    logging.basicConfig(filename = model_name+'.log',
                    level = logging.INFO,
                    format = '%(asctime)s:%(levelname)s:%(message)s')
 
    logging.info(sweep_configuration)
    
    # Prepare data
    test_loader, train_loader, val_loader = data(bs)
    
    # Build gnn model
    model = build_model(in_channels, hidden_dim_1, hidden_dim_2, hidden_dim_3, out_channels, nnconv_aggr)
    
    # Build optimizer
    optimizer = build_optimizer(model, lr)
    
    # Start training and validation process
    min_valid_loss = np.inf
    loss_values = []
    val_loss_values = []
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, epoch, lr, optimizer)
        #loss_values.append(train_loss)
        val_loss = evaluate_one_epoch(model, val_loader, epoch, lr)
        #val_loss_values.append(val_loss)
        wandb.log({
            'epoch': epoch, 
            'train_loss': train_loss, 
            'val_loss': val_loss
        })
# Start sweep job.
wandb.agent(sweep_id, function=main, count=15)

Create sweep with ID: l48o87j4
Sweep URL: https://wandb.ai/ntuwb/adam_mse_nnconv_poolMaxAggr_2fc_wandb_random_new/sweeps/l48o87j4


wandb: Agent Starting Run: xnfcgzeh with config:
wandb: 	batch_size: 96
wandb: 	criterion: MSELoss()
wandb: 	epochs: 60
wandb: 	hidden_dim_1: 64
wandb: 	hidden_dim_2: 128
wandb: 	hidden_dim_3: 32
wandb: 	in_channels: 11
wandb: 	lr: 0.004764430672401777
wandb: 	nnconv_aggr: mean
wandb: 	optimizer: adam
wandb: 	out_channels: 1


GNNModel(
  (conv1): NNConv(11, 64, aggr=mean, nn=Sequential(
    (0): Linear(in_features=5, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=704, bias=True)
  ))
  (global_pool): MaxAggregation()
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)
Epoch: 0, Training Loss: 1.2639
Epoch: 0, Validation Loss: 1.2144
Epoch: 1, Training Loss: 1.1796
Epoch: 1, Validation Loss: 1.1411
Epoch: 2, Training Loss: 1.0920
Epoch: 2, Validation Loss: 1.0450
Epoch: 3, Training Loss: 0.9929
Epoch: 3, Validation Loss: 0.9436
Epoch: 4, Training Loss: 0.9525
Epoch: 4, Validation Loss: 0.9516
Epoch: 5, Training Loss: 0.9276
Epoch: 5, Validation Loss: 0.8934
Epoch: 6, Training Loss: 0.9096
Epoch: 6, Validation Loss: 0.9139
Epoch: 7, Training Loss: 0.8995
Epoch: 7, Validation Loss: 0.9136
Epoch: 8, Training Loss: 0.8912
Epoch: 8, Validation Loss: 0.8535
Epoch: 9, Training Loss: 0.8871
Epoch: 9, Vali

wandb: Network error (ConnectionError), entering retry loop.


Epoch: 21, Training Loss: 0.8337
Epoch: 21, Validation Loss: 0.8368
Epoch: 22, Training Loss: 0.8269
Epoch: 22, Validation Loss: 0.8311
Epoch: 23, Training Loss: 0.8301
Epoch: 23, Validation Loss: 0.8551
Epoch: 24, Training Loss: 0.8230
Epoch: 24, Validation Loss: 0.8960
Epoch: 25, Training Loss: 0.8187
Epoch: 25, Validation Loss: 0.8164
Epoch: 26, Training Loss: 0.8184
Epoch: 26, Validation Loss: 0.8262
Epoch: 27, Training Loss: 0.8178
Epoch: 27, Validation Loss: 0.8272
Epoch: 28, Training Loss: 0.8162
Epoch: 28, Validation Loss: 0.8059
Epoch: 29, Training Loss: 0.8163
Epoch: 29, Validation Loss: 0.8278
Epoch: 30, Training Loss: 0.8131
Epoch: 30, Validation Loss: 0.8205
Epoch: 31, Training Loss: 0.8099
Epoch: 31, Validation Loss: 0.8294
Epoch: 32, Training Loss: 0.8083
Epoch: 32, Validation Loss: 0.8189
Epoch: 33, Training Loss: 0.8085
Epoch: 33, Validation Loss: 0.8191
Epoch: 34, Training Loss: 0.8053
Epoch: 34, Validation Loss: 0.8238
Epoch: 35, Training Loss: 0.8026
Epoch: 35, Vali

...

GNNModel(
  (conv1): NNConv(11, 64, aggr=add, nn=Sequential(
    (0): Linear(in_features=5, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=704, bias=True)
  ))
  (global_pool): MaxAggregation()
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)
Epoch: 0, Training Loss: 1.4400
Epoch: 0, Validation Loss: 1.2167
Epoch: 1, Training Loss: 1.2006
Epoch: 1, Validation Loss: 1.2177
Epoch: 2, Training Loss: 1.1827
Epoch: 2, Validation Loss: 1.1828
Epoch: 3, Training Loss: 1.1558
Epoch: 3, Validation Loss: 1.1632
Epoch: 4, Training Loss: 1.0564
Epoch: 4, Validation Loss: 1.0295
Epoch: 5, Training Loss: 1.0371
Epoch: 5, Validation Loss: 1.0424
Epoch: 6, Training Loss: 1.0201
Epoch: 6, Validation Loss: 1.0137
Epoch: 7, Training Loss: 1.0144
Epoch: 7, Validation Loss: 1.0074
Epoch: 8, Training Loss: 1.0023
Epoch: 8, Validation Loss: 1.0143
Epoch: 9, Training Loss: 1.0006
Epoch: 9, Valid

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▅▄▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_loss,██▇▇▃▃▃▃▃▂▂▂▂▁▁▁▂▁▂▁
epoch,19
train_loss,0.94839
val_loss,0.94563


wandb: Agent Starting Run: a4rgf5m9 with config:
wandb: 	batch_size: 128
wandb: 	criterion: MSELoss()
wandb: 	epochs: 40
wandb: 	hidden_dim_1: 64
wandb: 	hidden_dim_2: 128
wandb: 	hidden_dim_3: 32
wandb: 	in_channels: 11
wandb: 	lr: 0.06279269909246481
wandb: 	nnconv_aggr: max
wandb: 	optimizer: adam
wandb: 	out_channels: 1


GNNModel(
  (conv1): NNConv(11, 64, aggr=max, nn=Sequential(
    (0): Linear(in_features=5, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=704, bias=True)
  ))
  (global_pool): MaxAggregation()
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)
Epoch: 0, Training Loss: 9.5224
Epoch: 0, Validation Loss: 1.3043
Epoch: 1, Training Loss: 1.3172
Epoch: 1, Validation Loss: 1.3233
Epoch: 2, Training Loss: 1.3068
Epoch: 2, Validation Loss: 1.3416
Epoch: 3, Training Loss: 1.3019
Epoch: 3, Validation Loss: 1.2866
Epoch: 4, Training Loss: 1.2981
Epoch: 4, Validation Loss: 1.2893
Epoch: 5, Training Loss: 1.2976
Epoch: 5, Validation Loss: 1.2743
Epoch: 6, Training Loss: 1.3129
Epoch: 6, Validation Loss: 1.2763
Epoch: 7, Training Loss: 1.2949
Epoch: 7, Validation Loss: 1.2915
Epoch: 8, Training Loss: 1.2960
Epoch: 8, Validation Loss: 1.3340
Epoch: 9, Training Loss: 1.2999
Epoch: 9, Valid

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▃▃▁▂▁▁▂▃▂▂▂▂▂▁▅▁▃▆▁▂▃▂▂▃▂▃▂▄█▄▆▂▂▂▂▂▂▂▂
epoch,39
train_loss,1.31752
val_loss,1.29654


In [ ]:
np.savetxt('./loss_values/loss_values_'+model_name+'.txt',loss_values)
np.savetxt('./loss_values/val_loss_values_'+model_name+'.txt',val_loss_values)

By defining the message() and update() functions, you control how information is propagated through the graph and how nodes update their representations based on the received messages. These functions enable the GNN to learn from the graph structure and capture useful information for the prediction task.

During the forward() pass, the GNN iteratively performs message passing and aggregation steps across the graph, combining information from neighboring nodes to update each node's representation. The output of the forward() function provides the final representations that can be further processed or used for downstream tasks.

In [ ]:
# Test after training
model.eval()
total_test_loss = 0

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        output = model(data).flatten()
        test_loss = config["criterion"](output, data.y)
        total_test_loss += test_loss.item() * data.num_graphs

average_test_loss = total_test_loss / len(test_loader.dataset)
print(f'Test Loss before training: {average_test_loss_before:.4f}')
print(f'Test Loss after training: {average_test_loss:.4f}')
logging.info(f'Test Loss before training: {average_test_loss_before:.4f}')
logging.info(f'Test Loss after training: {average_test_loss:.4f}')

In [ ]:
loss_values = np.loadtxt("./loss_values/loss_values_"+model_name+".txt")
valid_loss_values = np.loadtxt("./loss_values/val_loss_values_"+model_name+".txt")

In [ ]:
plt.plot(loss_values, label='train loss:'+model_name, ls='-', ms=20, markevery=100, alpha=0.5)
plt.plot(valid_loss_values,'y', label='validation loss:'+model_name, ls='-', ms=20, markevery=100, alpha=0.8)

#don't know why the markevery doesn't work
plt.xlabel('epoch') 
plt.ylabel('loss') 
plt.title("Loss")
plt.legend()
plt.show()

In [29]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(11, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=1, bias=True)
)
